<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/friedmann_hz_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Friedmann H(z) simulator

Computes the cosmological expansion rate H(z) in ΛCDM (optionally including radiation and curvature):
    H(z) = H0 * sqrt(Ω_r (1+z)^4 + Ω_m (1+z)^3 + Ω_k (1+z)^2 + Ω_Λ)

Units:
- H0: km/s/Mpc
- z: dimensionless redshift (z >= 0 recommended)
- Ω parameters: dimensionless density fractions

Features:
- Single evaluation or parameter sweeps via CLI
- CSV export
- Optional plotting of H(z) or E(z)=H(z)/H0
- Notebook/Colab compatible (sanitizes [-f, kernel.json] args)
"""

import argparse
import csv
import sys
from typing import List, Optional, Sequence, Tuple

import numpy as np


# -----------------------
# Argument sanitization (Notebook/Colab)
# -----------------------

def sanitize_argv(argv: Sequence[str]) -> List[str]:
    """
    Remove Jupyter/Colab-injected args like '-f <kernel.json>'.
    """
    cleaned: List[str] = []
    skip_next = False
    for i, a in enumerate(argv):
        if skip_next:
            skip_next = False
            continue
        if a == "-f":
            skip_next = True
            continue
        if a.endswith(".json") and "jupyter/runtime/kernel" in a:
            continue
        cleaned.append(a)
    return cleaned


# -----------------------
# Cosmology core
# -----------------------

def compute_omega_k(omega_m: float, omega_lambda: float, omega_r: float) -> float:
    """
    If curvature is 'auto', set Ω_k = 1 - (Ω_m + Ω_Λ + Ω_r).
    """
    return 1.0 - (omega_m + omega_lambda + omega_r)


def E_of_z(z: np.ndarray,
           omega_m: float,
           omega_lambda: float,
           omega_r: float,
           omega_k: float) -> np.ndarray:
    """
    Dimensionless expansion rate E(z) = H(z)/H0.
    """
    zp1 = 1.0 + z
    return np.sqrt(
        omega_r * zp1**4 +
        omega_m * zp1**3 +
        omega_k * zp1**2 +
        omega_lambda
    )


def H_of_z(H0: float,
           z: np.ndarray,
           omega_m: float,
           omega_lambda: float,
           omega_r: float = 0.0,
           omega_k: Optional[float] = None,
           curvature_mode: str = "flat") -> Tuple[np.ndarray, float]:
    """
    Compute H(z) and the Ω_k actually used.

    curvature_mode:
      - "flat": force Ω_k = 0
      - "auto": Ω_k = 1 - (Ω_m + Ω_Λ + Ω_r)
      - "manual": use provided omega_k (default 0.0 if None)
    """
    if curvature_mode not in {"flat", "auto", "manual"}:
        raise ValueError("curvature_mode must be one of: flat, auto, manual")

    if curvature_mode == "flat":
        ok = 0.0
    elif curvature_mode == "auto":
        ok = compute_omega_k(omega_m, omega_lambda, omega_r)
    else:  # manual
        ok = 0.0 if omega_k is None else float(omega_k)

    Ez = E_of_z(np.asarray(z, dtype=float), omega_m, omega_lambda, omega_r, ok)
    return H0 * Ez, ok


# -----------------------
# IO helpers
# -----------------------

def save_csv(path: str, header: List[str], rows: np.ndarray) -> None:
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(header)
        w.writerows(rows)


def try_plot(x: np.ndarray,
             y: np.ndarray,
             xlabel: str,
             ylabel: str,
             title: str,
             out_path: Optional[str],
             show: bool) -> None:
    try:
        import matplotlib.pyplot as plt
    except Exception:
        print("Plotting skipped: matplotlib not available.", file=sys.stderr)
        return
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot(x, y, lw=2)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    fig.tight_layout()
    if out_path:
        fig.savefig(out_path, dpi=160)
        print(f"Saved plot to {out_path}")
    if show and not out_path:
        plt.show()
    plt.close(fig)


# -----------------------
# CLI handlers
# -----------------------

def handle_single(args: argparse.Namespace) -> None:
    z = float(args.z)
    if z < -0.999999:
        raise ValueError("z must be >= -0.999999 (scale factor a > 0).")
    Hvals, omega_k_used = H_of_z(
        H0=args.H0,
        z=np.array([z], dtype=float),
        omega_m=args.omega_m,
        omega_lambda=args.omega_lambda,
        omega_r=args.omega_r,
        omega_k=args.omega_k,
        curvature_mode=args.curvature
    )
    H = float(Hvals[0])
    E = H / args.H0
    print("Friedmann single evaluation")
    print(f"  H0           : {args.H0:.6f} km/s/Mpc")
    print(f"  z            : {z:.6f}")
    print(f"  Ω_m, Ω_Λ, Ω_r: {args.omega_m:.6f}, {args.omega_lambda:.6f}, {args.omega_r:.6f}")
    print(f"  curvature    : {args.curvature} (Ω_k = {omega_k_used:.6f})")
    if args.normalize:
        print(f"  E(z)=H/H0    : {E:.8f}")
    else:
        print(f"  H(z)         : {H:.8f} km/s/Mpc")


def handle_sweep(args: argparse.Namespace) -> None:
    z0, z1, n = args.z_range
    n = int(n)
    if n < 2:
        raise ValueError("N in --z-range must be >= 2")
    if z0 < -0.999999:
        raise ValueError("Start z must be >= -0.999999 (scale factor a > 0).")

    z = np.linspace(z0, z1, n, dtype=float)
    H, omega_k_used = H_of_z(
        H0=args.H0,
        z=z,
        omega_m=args.omega_m,
        omega_lambda=args.omega_lambda,
        omega_r=args.omega_r,
        omega_k=args.omega_k,
        curvature_mode=args.curvature
    )
    E = H / args.H0

    # CSV
    if args.out_csv:
        rows = np.column_stack([z, H, E])
        save_csv(args.out_csv, ["z", "H_km_per_s_per_Mpc", "E=H_over_H0"], rows)
        print(f"Saved CSV: {args.out_csv} ({rows.shape[0]} rows)")

    # Plot
    if args.plot:
        y = E if args.normalize else H
        ylabel = "E(z) = H/H0" if args.normalize else "H(z) [km/s/Mpc]"
        title = f"Expansion rate vs z (curvature={args.curvature}, Ω_k={omega_k_used:.4f})"
        try_plot(z, y, "Redshift z", ylabel, title, args.plot_out, args.show)

    # Preview
    print("Preview (first 5):")
    for zi, Hi, Ei in zip(z[:5], H[:5], E[:5]):
        print(f"  z={zi:.6f}  H={Hi:.6f} km/s/Mpc  E={Ei:.6f}")


# -----------------------
# Parser
# -----------------------

def add_cosmo_args(p: argparse.ArgumentParser) -> None:
    p.add_argument("--H0", type=float, required=True, help="Hubble constant H0 (km/s/Mpc), e.g., 70")
    p.add_argument("--omega-m", type=float, default=0.3, help="Ω_m (matter density), default 0.3")
    p.add_argument("--omega-lambda", type=float, default=0.7, help="Ω_Λ (dark energy density), default 0.7")
    p.add_argument("--omega-r", type=float, default=0.0, help="Ω_r (radiation density), default 0.0")
    p.add_argument("--omega-k", type=float, default=None,
                   help="Ω_k (curvature). Ignored if --curvature=flat or --curvature=auto.")
    p.add_argument("--curvature", choices=["flat", "auto", "manual"], default="flat",
                   help="Curvature mode: flat (Ω_k=0), auto (Ω_k=1-Ω_tot), manual (use --omega-k). Default: flat")
    p.add_argument("--normalize", action="store_true", help="Output/plot E(z)=H/H0 instead of H(z)")


def build_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(
        prog="friedmann_hz.py",
        description="Friedmann equation H(z) simulator (single-run and sweeps), notebook-safe."
    )
    sub = parser.add_subparsers(dest="cmd", help="Commands")

    # single
    s = sub.add_parser("single", help="Compute H(z) (or E(z)) at one redshift.")
    add_cosmo_args(s)
    s.add_argument("--z", type=float, required=True, help="Redshift z (>= 0 recommended)")
    s.set_defaults(func=handle_single)

    # sweep
    w = sub.add_parser("sweep", help="Compute H(z) (or E(z)) over a redshift range.")
    add_cosmo_args(w)
    w.add_argument("--z-range", type=float, nargs=3, metavar=("START", "STOP", "N"),
                   required=True, help="Redshift linspace: START STOP N (e.g., 0 10 200)")
    w.add_argument("--out-csv", type=str, default=None, help="Path to save CSV (z, H, E).")
    w.add_argument("--plot", action="store_true", help="Generate a plot.")
    w.add_argument("--plot-out", type=str, default=None, help="Optional path to save plot image.")
    w.add_argument("--show", action="store_true", help="Show the plot window if available (mostly local use).")
    w.set_defaults(func=handle_sweep)

    return parser


# -----------------------
# Main (Notebook-safe)
# -----------------------

def main(argv: Optional[Sequence[str]] = None) -> None:
    if argv is None:
        argv = sys.argv[1:]
    argv = sanitize_argv(argv)

    parser = build_parser()
    # Do NOT make subparsers required; print help when missing instead of SystemExit
    if len(argv) == 0:
        parser.print_help()
        return

    try:
        args = parser.parse_args(argv)
    except SystemExit:
        # Honor argparse behavior on real CLI; in notebooks it's better to let it surface
        raise

    # Dispatch
    handler = getattr(args, "func", None)
    if handler is None:
        parser.print_help()
        return
    handler(args)


if __name__ == "__main__":
    main()